In [334]:
import numpy as np
import mysql.connector as connection
import pandas as pd
from scipy.stats import zscore
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score, cross_val_predict,ShuffleSplit
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score
from joblib import dump, load
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as msa

import category_encoders as ce
import catboost as cb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

In [340]:
años = [2015,2016,2017,2018,2019,2020,2021]
lista_db = []
try:
    mydb = connection.connect(host="192.168.0.26", database = 'root',user="mlab1", passwd="",use_pure=True)
    for año in años:
        query = f"Select * from muestra_{año};"
        lista_db.append(pd.read_sql(query,mydb))
    mydb.close() #close the connection
except Exception as e:
    mydb.close()
    print(str(e))

1045 (28000): Access denied for user 'mlab1'@'DESKTOP-I2RB40E' (using password: NO)


In [336]:
db_filtrada = []
for db in lista_db:
    db_filtrada.append(db[[ 'ph', 'n', 'mo', 'k', 'ca', 'mg','al',"cic"]])

In [337]:
df = pd.concat(db_filtrada)


# Aluminio

In [272]:
df_al = df[[ 'ph', 'n', 'mo', 'k', 'ca', 'mg','al']]

In [273]:
df_al = df_al.dropna()

In [274]:
cols = df_al.columns.values
df_al = df_al.replace(",",".",regex=True)




In [277]:
df_al[cols] = df_al[cols].apply(pd.to_numeric, errors='coerce', axis=1)
df_al = df_al.dropna()

In [278]:

from sklearn.preprocessing import RobustScaler

X = df_al[['ph',"n",'mo', 'k', 'ca', 'mg']]
y = df_al["al"]

sc = StandardScaler()
train_cbe = sc.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(train_cbe, y, test_size=0.2, random_state=42)



In [279]:
# check xgboost version
import xgboost as xg

In [282]:
model =   xg.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=2, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model.fit(X_train, y_train)

[11:32:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.4603,
             enable_categorical=False, gamma=0.0468, gpu_id=-1,
             importance_type=None, interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=2,
             min_child_weight=1.7817, missing=nan, monotone_constraints='()',
             n_estimators=2200, n_jobs=8, nthread=-1, num_parallel_tree=1,
             predictor='auto', random_state=7, reg_alpha=0.464,
             reg_lambda=0.8571, scale_pos_weight=1, silent=1, subsample=0.5213,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [288]:
from sklearn.metrics import mean_absolute_error
pred = model.predict(X_test)
rmse = (mean_absolute_error(y_test, abs(pred)))
r2 = r2_score(y_test, pred)
print("Testing performance")
print("MAE: {:.2f}".format(rmse))
print("R2: {:.2f}".format(r2))

Testing performance
MAE: 0.50
R2: 0.60


In [289]:
test_df = pd.read_excel("calcular.xlsx")

In [290]:
test_df

,Tipo Análisis,N. Lab,pH,N,MO,K,Ca,Mg,Al,CIC,P,S,B
0,C III,8032,5.2,0.44,11.0,0.12,0.64,0.27,NaN,NaN,12,12.4,0.21
1,C III,8033,5.3,0.39,9.7,0.16,0.80,0.36,NaN,NaN,11,12.5,0.06
2,CIII - sin TB - CICE,8047,4.0,0.32,7.5,0.59,2.25,0.62,NaN,NaN,182,10.2,0.89
3,CIII - sin TB - CICE,8048,4.1,0.35,8.4,0.55,1.20,0.51,NaN,NaN,64,30.2,0.99
4,CIII - sin TB - CICE,8049,5.2,0.31,7.3,0.55,23.19,0.77,NaN,NaN,183,27.8,1.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,C III,8308,5.3,0.30,6.9,0.66,3.94,1.14,NaN,NaN,95,15.0,NaN
257,C III,8309,5.1,0.29,6.8,0.66,6.04,1.37,NaN,NaN,94,11.8,NaN
258,C III,8310,4.4,0.25,5.6,0.60,4.88,1.22,NaN,NaN,256,13.1,NaN
259,C III,8311,5.0,0.26,5.9,0.56,4.79,1.45,NaN,NaN,150,9.9,NaN


In [301]:

test_df = test_df[[ 'pH',"N" ,'MO', 'K', 'Ca', 'Mg',"Al"]]
X = test_df[[ 'pH', 'N',"MO",'K', 'Ca', 'Mg',]]
X = sc.transform(X)
pred = abs(np.round(model.predict(X),2))


C:\Users\carlo\anaconda3\lib\site-packages\sklearn\base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Ca
- K
- MO
- Mg
- N
- ...
Feature names seen at fit time, yet now missing:
- ca
- k
- mg
- mo
- n
- ...

  warnings.warn(message, FutureWarning)


In [307]:
al = pred

# CIC

In [309]:
df_cic = df[[ 'ph', 'n', 'mo', 'k', 'ca', 'mg','cic']]

In [310]:
cols = df_cic.columns.values
df_cic = df_cic.replace(",",".",regex=True)

In [311]:
df_cic[cols] = df_cic[cols].apply(pd.to_numeric, errors='coerce', axis=1)
df_cic = df_cic.dropna()

In [312]:
X = df_cic[['ph',"n", 'mo', 'k', 'ca', 'mg']]
y = df_cic["cic"]

In [313]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import GridSearchCV


sc = StandardScaler()
train_cbe = sc.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(train_cbe, y,
test_size=0.2,random_state=42)

model =   xg.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=2, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model.fit(X_train, y_train)
pred = model.predict(X_test)
rmse = (np.sqrt(mean_squared_error(y_test, pred)))
r2 = r2_score(y_test, pred)
print("Testing performance")
print("RMSE: {:.2f}".format(rmse))
print("R2: {:.2f}".format(r2))

[11:35:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


Testing performance
RMSE: 5.03
R2: 0.55


In [317]:
test_df = pd.read_excel("calcular.xlsx")
test_df = test_df[[ 'pH',"N" ,'MO', 'K', 'Ca', 'Mg',"CIC"]]
X = test_df[[ 'pH', 'N',"MO",'K', 'Ca', 'Mg',]]
y = test_df[["CIC"]]
test = sc.transform(X)
pred = model.predict(test)


C:\Users\carlo\anaconda3\lib\site-packages\sklearn\base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Ca
- K
- MO
- Mg
- N
- ...
Feature names seen at fit time, yet now missing:
- ca
- k
- mg
- mo
- n
- ...

  warnings.warn(message, FutureWarning)


In [318]:
cic = pred

In [319]:
cic

array([25.736338 , 22.936779 , 18.909615 , 20.112867 , 22.11241  ,
       23.038704 , 27.61392  , 18.291918 , 19.756014 , 21.465189 ,
       17.53953  , 23.736973 , 21.063545 , 26.674377 , 20.286009 ,
       17.057583 , 19.868868 , 16.66818  , 17.89805  , 19.82712  ,
       26.367758 , 15.932736 , 17.218136 , 18.42927  , 18.42964  ,
       19.661022 , 16.532043 , 16.443808 , 19.407677 , 16.06585  ,
       19.000885 , 16.497786 , 26.636717 , 28.012798 , 23.559158 ,
       17.82755  , 18.699574 , 24.18783  , 28.610977 , 21.411402 ,
       25.287348 , 22.288578 , 24.117287 , 19.179388 , 21.043219 ,
       25.398218 , 22.218292 , 25.99528  , 18.505072 , 16.54908  ,
       24.72383  , 19.91671  , 18.853075 , 18.08363  , 20.047928 ,
       15.727742 , 14.587927 , 17.569626 , 14.155244 , 16.795824 ,
       16.273714 , 16.501595 , 15.142033 , 14.979241 , 13.677252 ,
       16.041828 , 16.947035 , 17.285019 , 14.016077 , 14.660098 ,
       15.139967 , 14.778065 , 16.091785 , 18.40602  , 18.2352

In [330]:
# Guardar

test_df = pd.read_excel("calcular.xlsx")

In [331]:
test_df["Al"] = al
test_df["CIC"] = np.round(cic,2)

In [332]:
test_df

,Tipo Análisis,N. Lab,pH,N,MO,K,Ca,Mg,Al,CIC,P,S,B
0,C III,8032,5.2,0.44,11.0,0.12,0.64,0.27,0.64,25.740000,12,12.4,0.21
1,C III,8033,5.3,0.39,9.7,0.16,0.80,0.36,0.64,22.940001,11,12.5,0.06
2,CIII - sin TB - CICE,8047,4.0,0.32,7.5,0.59,2.25,0.62,2.93,18.910000,182,10.2,0.89
3,CIII - sin TB - CICE,8048,4.1,0.35,8.4,0.55,1.20,0.51,3.34,20.110001,64,30.2,0.99
4,CIII - sin TB - CICE,8049,5.2,0.31,7.3,0.55,23.19,0.77,0.10,22.110001,183,27.8,1.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,C III,8308,5.3,0.30,6.9,0.66,3.94,1.14,0.21,19.870001,95,15.0,NaN
257,C III,8309,5.1,0.29,6.8,0.66,6.04,1.37,0.22,19.389999,94,11.8,NaN
258,C III,8310,4.4,0.25,5.6,0.60,4.88,1.22,1.63,17.520000,256,13.1,NaN
259,C III,8311,5.0,0.26,5.9,0.56,4.79,1.45,0.35,17.959999,150,9.9,NaN


In [333]:
test_df.to_excel("calculado.xlsx")